In [1]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from time import sleep
from lxml import etree
import pandas as pd
from selenium.webdriver.common.by import By  
import  requests


In [ ]:
# cd C:\Users\yuhuyi\AppData\Local\Google\Chrome\Application\
# chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\selenium\ChromeProfile"
# https://sycm.taobao.com/mc/free/market_rank

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
bro = webdriver.Chrome(options=chrome_options)
window_handles = bro.window_handles
bro.switch_to.window(window_handles[0])
bro.execute_script("window.scrollTo(0,document.body.scrollHeight)")
bro.maximize_window()

In [ ]:
A = bro.find_element(By.XPATH,'//button[@class="ecom-pagination-item-link"]')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//body/script"}
  (Session info: chrome=99.0.4844.51)
Stacktrace:
Backtrace:
	Ordinal0 [0x010D9943+2595139]
	Ordinal0 [0x0106C9F1+2148849]
	Ordinal0 [0x00F64528+1066280]
	Ordinal0 [0x00F90FD4+1249236]
	Ordinal0 [0x00F911CB+1249739]
	Ordinal0 [0x00FBD812+1431570]
	Ordinal0 [0x00FABA34+1358388]
	Ordinal0 [0x00FBBAF2+1424114]
	Ordinal0 [0x00FAB806+1357830]
	Ordinal0 [0x00F86086+1204358]
	Ordinal0 [0x00F86F96+1208214]
	GetHandleVerifier [0x0127B232+1658114]
	GetHandleVerifier [0x0133312C+2411516]
	GetHandleVerifier [0x0116F261+560433]
	GetHandleVerifier [0x0116E366+556598]
	Ordinal0 [0x0107286B+2173035]
	Ordinal0 [0x010775F8+2192888]
	Ordinal0 [0x010776E5+2193125]
	Ordinal0 [0x010811FC+2232828]
	BaseThreadInitThunk [0x7790FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77A6809E+286]
	RtlGetAppContainerNamedObjectPath [0x77A6806E+238]


In [9]:
html_str = bro.page_source
HtmlData = etree.HTML(html_str)

In [ ]:
def get_data(html_str):
    data = pd.DataFrame(columns=["店铺名称","成交金额","点击次数","商品名称","价格区间","排名"])
    HtmlData = etree.HTML(html_str)
    tr_list = HtmlData.xpath("//tr[@class='ecom-table-row ecom-table-row-level-0']")
    for tr in tr_list:  
        info_dicts = {}
        info_dicts["店铺名称"] = tr.xpath(".//div[@class='name-qfS2bZ']/text()")[0]
        info_dicts["成交金额"] = tr.xpath(".//div[@class='numText-qvEDO1']/text()")[0]
        info_dicts["点击次数"] = tr.xpath(".//div[@class='numText-qvEDO1']/text()")[1]
        info_dicts["商品名称"] = tr.xpath(".//div[@class='name-olPpkd']/text()")[0]
        info_dicts["排名"] = tr.xpath(".//div[@class='rank-uKOWOn']/text()")[0]
        info_dicts["价格区间"] = tr.xpath(".//div[@class='price-T7iKi4']/span/text()")[0]
        info_df = pd.DataFrame(info_dicts,index=[0])
        data = pd.concat([data,info_df])
    return data

In [ ]:
need_data = pd.DataFrame(columns=["店铺名称","成交金额","点击次数","商品名称","价格区间","排名"])
i = 1
while i < 21:
    print(i)
    page = bro.page_source
    sleep(1)
    datasss=get_data(page)
    sleep(1)
    need_data = pd.concat([need_data,datasss])
    bro.execute_script("arguments[0].click()", A)
    i=i+1
    
else:
    # page = bro.page_source
    # sleep(1)
    # datasss=get_data(page)
    # sleep(1)
    # need_data = pd.concat([need_data,datasss])
    pass

In [ ]:
pd.set_option('display.max_rows', None)
need_data.drop_duplicates(inplace=True)
need_data

In [ ]:
bro.page_source

In [ ]:
# C = bro.find_element(By.XPATH,'//span[@class="anticon anticon-left"]')
# bro.execute_script("arguments[0].click()", C)

In [ ]:
need_data.drop_duplicates(inplace=True)

In [ ]:
need_data.to_excel(".\spider数据.xlsx",index=False)